<a href="https://colab.research.google.com/github/pneuly/whisper-asr-colab/blob/develop/whisper_asr_colab_ja.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import logging
import locale
import functools
from google.colab import files

locale.getpreferredencoding = lambda: "UTF-8"
os.environ["UV_PRERELEASE"] = "if-necessary-or-explicit"
os.environ["UV_SYSTEM_PYTHON"] = "true"
os.environ["UV_NO_PROGRESS"] = "true"
for var in ["UV_BUILD_CONSTRAINT", "UV_CONSTRAINT"]:
    os.environ.pop(var, None)

# Avoid unnecessary downgrades
with open("override.txt", "w") as f:
    f.writelines("""nvidia-cudnn-cu12
nvidia-cublas-cu12
nvidia-cuda-cupti-cu12
nvidia-cuda-nvrtc-cu12
nvidia-cuda-runtime-cu12
nvidia-cufft-cu12
nvidia-curand-cu12
nvidia-cusolver-cu12
nvidia-cusparse-cu12
nvidia-nvjitlink-cu12
nvidia-nccl-cu12
numpy
opentelemetry-api
opentelemetry-sdk
opentelemetry-semantic-conventions
opentelemetry-exporter-otlp-proto-common
opentelemetry-exporter-otlp-proto-http
opentelemetry-proto
""")
os.environ["UV_OVERRIDE"] = "override.txt"

# @title 自動文字起こし（[Source Code](https://github.com/pneuly/whisper-asr-colab)）{ display-mode: "form" }
# @markdown <font color="red">以下の設定項目を入力しセルを実行（Ctrl+Enter）</font><br>
# ※<b>正常に文字起こしができない場合はstart_time、end_timeを指定して音声を分割して下さい</b>
# @markdown <b>（設定項目の説明は下にあります）</b></br>

audio = 'https://www.youtube.com/watch?v=xAmEQOqtMvA'  # @param {type:"string"}
password = ""  # @param {type:"string"}
download_format = "mp3"  # @param ["", "mp3", "m4a", "aac", "vorbis", "opus", "wav"] {allow-input: true}
model_size = "large-v3-turbo" # @param ["large-v3-turbo", "large-v3", "large-v2", "large", "medium", "small", "base", "tiny"] {allow-input: true}
diarization = True  # @param {type:"boolean"}
start_time = ""  # @param {type:"string"}
end_time = ""  # @param {type:"string"}
timestamp_offset = ""  # @param {type:"string"}
skip_silence = True  # @param {type:"boolean"}
hotwords = "次に、これです。" #  {type:"string"}
batch_size = 1 #  {type:"number"}
realtime = False  #  {type:"boolean"}
HUGGING_FACE_TOKEN = ""  # @param {type:"string"}
initial_prompt = "定刻になりましたので、開始いたします。"
prefix = None
vad_filter = False
files_to_download = []

# @markdown ###<br/><b>〔設定の説明〕</b>
# @markdown <b>audio:</b> 文字起こしする音声ファイルの場所<br/>
# @markdown 　（例）https://www.youtube.com/...... ：Youtube<br/>
# @markdown　　　　230401_1010.mp3 ：手動でアップロードしたファイル<br/>
# @markdown　　　　<font color="red">空欄にするとアップロードボタンが表示されます。リリースフォルダからファイルをアップロードしてください</font>
# @markdown <br/><b>download_format:</b> Youtube等からダウンロードする際の音声ファイル形式
# @markdown <br/><b>model_size:</b> 音声認識のモデルサイズ
# @markdown <br/><b>diarization:</b> 発言者別の文字起こしファイルを作成するか
# @markdown #### <br/><b><font color= "blue">以下は必要な場合のみ設定</font></b>
# @markdown <br/><b>password:</b> 音声DL時のパスワード（Webexなど）/音声をパスワード付きzipでアップロードした場合のzipパスワード</b>
# @markdown <br/><b>start_time:</b> 開始時間 秒 or hh:mm:ss</b>（指定しない場合は最初から）
# @markdown <br/><b>end_time:</b> 終了時間 秒 or hh:mm:ss（指定しない場合は最後まで）
# @markdown <br/><b>timestamp_offset:</b> タイムスタンプを指定の時間だけずらす 秒 or hh:mm:ss（指定しない場合はstart_timeと連動）
# @markdown <br/><b>skip_silence:</b> 先頭と最後の無音区間を削除
## @markdown <br/><b>hotwords:</b> キーワード（次に、これです。は句読点を付けるために入れています。）
## @markdown <br/><b>batch_size:</b> 2以上にすると少し早くなります。1の場合と2以上の場合で文字起こし結果が変わります。
## @markdown <br/><b>reatime: </b><font color="red">ストリーミングをリアルタイムで文字起こしをする場合のみオンにしてください。</font>

# ----- main routine ------
logging.getLogger().setLevel(logging.WARNING)
logger = logging.getLogger(__name__)
#logger.setLevel(logging.DEBUG)
if not logger.hasHandlers():
    logger.addHandler(logging.StreamHandler())

if diarization and HUGGING_FACE_TOKEN == "":
    raise ValueError("HUGGING_FACE_TOKEN にトークンを入力してください")

def _transcribe(audio):
    from whisper_asr_colab.asr import ASRWorker
    worker = ASRWorker(
        audio=audio,
        load_model_args={"model_size_or_path" : model_size,},
        transcribe_args={
            "language" : "ja",
            "initial_prompt" : initial_prompt,
            "batch_size" : batch_size,
            "hotwords" : hotwords,
            "prefix" : prefix,
            "vad_filter" : False,}
    )

    if timestamp_offset:
        worker.timestamp_offset = timestamp_offset
    results = worker.run()
    # gc GPU RAM
    del worker
    return results


def _diarize():
    !uv pip install pyannote.audio
    from whisper_asr_colab.diarize import DiarizationWorker # noqa: E402
    from torch.cuda import empty_cache
    empty_cache()
    diarizer = DiarizationWorker(
        audio=audio,
        hugging_face_token = HUGGING_FACE_TOKEN,
        )
    return diarizer.run()


def _convert_audio_if_needed():
    from whisper_asr_colab.audio import convert_audio
    return convert_audio(audio.local_file_path, download_format)



if __name__ == "__main__":
    !uv pip install git+https://github.com/pneuly/whisper-asr-colab.git@develop

    from whisper_asr_colab.audio import Audio # noqa: E402
    from whisper_asr_colab.common import unzip_with_password, process_isolator
    from whisper_asr_colab.asr import ASR_PROGRESS_FILE
    from faster_whisper.utils import download_model

    if audio == "":
        audio = list(files.upload())[0]

    if isinstance(audio, str) and audio.endswith('.zip'):
        audio = unzip_with_password(audio, password)[0]

    audio = Audio(
        source = audio,
        start_time=start_time,
        end_time=end_time,
        download_format=download_format,
        password=password,
    )

    audio.verify_upload = False
    audio._load_audio()

    download_model(model_size)

    result = process_isolator(
        _transcribe,
        ASR_PROGRESS_FILE,
        audio,
    )

    files_to_download.extend(result.values())

    if diarization:
        diarized_txt = _diarize()
        files_to_download.append(diarized_txt)

        print("Writing to docx.")
        from whisper_asr_colab.docx_generator import DocxGenerator
        doc = DocxGenerator()
        doc.txt_to_word(diarized_txt)
        files_to_download.append(doc.docfilename)

    if audio.is_remote:
        # Add audio file to files_to_download
        audio_file = _convert_audio_if_needed()
        files_to_download.append(audio_file)

    # Download files
    from whisper_asr_colab.common.utils import download_from_colab
    for file in files_to_download:
        print(f"Downloading {file}")
        download_from_colab(file)
